# Задание по программированию: Логистическая регрессия

## Вы научитесь:
- работать с логистической регрессией
- реализовывать градиентный спуск для ее настройки
- использовать регуляризацию

## Введение
Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность **оценивания вероятностей** классов, тогда как большинство линейных классификаторов могут выдавать только номера классов.

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Мы будем работать с выборкой, содержащей два признака. Будем считать, что ответы лежат в множестве {-1, 1}. Для настройки логистической регрессии мы будем решать следующую задачу:

<img src="img/1.png">

Здесь $x_{i1}$ и $x_{i2}$ — значение первого и второго признаков соответственно на объекте $x_i$. В этом задании мы будем рассматривать алгоритмы без свободного члена, чтобы упростить работу.

Градиентный шаг для весов будет заключаться в одновременном обновлении весов $w_1$ и $w_2$ по следующим формулам (проверьте сами, что здесь действительно выписана производная нашего функционала):

<img src="img/2.png">

Здесь k — размер шага.

Линейные методы могут переобучаться и давать плохое качество из-за различных проблем в данных: мультиколлинеарности, зашумленности и т.д. Чтобы избежать этого, следует использовать регуляризацию — она позволяет понизить сложность модели и не допустить переобучения. Сила регуляризации определяется коэффициентом C в формулах, указанных выше.

## Реализация в Scikit-Learn
В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск.

В качестве метрики качества будем использовать AUC-ROC (Area Under ROC-Curve). Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

В Scikit-Learn метрика AUC реализована функцией sklearn.metrics.roc_auc_score. В качестве первого аргумента ей передается вектор истинных ответов, в качестве второго — вектор с оценками принадлежности объектов к первому классу.

## Материалы
- <a href="https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture05-linclass.pdf">Подробнее о логистической регрессии и предсказании вероятностей с ее помощью</a>
- <a href="https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture02-linregr.pdf">Подробнее о градиентах и градиентном спуске</a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

## Инструкция по выполнению
1) Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [2]:
data = pd.read_csv('data/data-logistic.csv', header=None, names=['y', 'x1', 'x2'])

In [3]:
data.head()

,y,x1,x2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
y     205 non-null int64
x1    205 non-null float64
x2    205 non-null float64
dtypes: float64(2), int64(1)
memory usage: 4.9 KB


2) Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

3) Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [5]:
def step(data, w, C=10, k=0.1):
    sum1 = 0
    sum2 = 0
    for i in range(len(data)):
        d = 1 - (1/(1 + np.exp(-data.y[i] * (w[0] * data.x1[i] + w[1] * data.x2[i]))))
        sum1 += data.y[i] * data.x1[i] * d
        sum2 += data.y[i] * data.x2[i] * d
        
    w1_new = w[0] + k * (1./len(data)) * sum1 - k * C * w[0]
    w2_new = w[1] + k * (1./len(data)) * sum2 - k * C * w[1]
    
    return (w1_new, w2_new)

4) Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [6]:
def grad_desc(data, w0=(0, 0), eps=1e-5, max_iter=10000, C=10, k=0.1):
    w_old = w0
    n_iter = 0
    delta = 2 * eps
    while (delta > eps and n_iter < max_iter):
        w_new = step(data, w_old, C=C, k=k)
        delta = np.sqrt((w_new[0] - w_old[0])**2 + (w_new[1] - w_old[1])**2)
        w_old = w_new
        n_iter += 1
        
    return w_new

In [7]:
%%time
w = grad_desc(data)
print(w)

(0.028558754546234223, 0.024780137249735559)
Wall time: 413 ms


5) Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

In [8]:
# функция для записи ответов
def write_answer(name, answer):
    with open('data/' + name + '.txt', 'w') as file:
        file.write(str(answer))

In [9]:
# функция для оценки вероятности принадлежности к классу "+1"
def a(data, w):
    return 1 / (1 + np.exp(-w[0] * data.x1 - w[1] * data.x2 ))

In [10]:
data['y_predict_C=10'] = a(data, w=w)

In [11]:
data.head()

,y,x1,x2,y_predict_C=10
0,-1,-0.663827,-0.138526,0.494403
1,1,1.994596,2.468025,0.529496
2,-1,-1.247395,0.749425,0.495737
3,1,2.309374,1.899836,0.528228
4,1,0.849143,2.407750,0.520966


In [12]:
%%time
w_0 = grad_desc(data, C=0)
print(w_0)

(0.28781162047177639, 0.09198330215925439)
Wall time: 9.96 s


In [13]:
data['y_predict_C=0'] = a(data, w=w_0)

In [14]:
data.head()

,y,x1,x2,y_predict_C=10,y_predict_C=0
0,-1,-0.663827,-0.138526,0.494403,0.449226
1,1,1.994596,2.468025,0.529496,0.690206
2,-1,-1.247395,0.749425,0.495737,0.427984
3,1,2.309374,1.899836,0.528228,0.698343
4,1,0.849143,2.407750,0.520966,0.614405


In [15]:
auc = [str(round(roc_auc_score(data.y.values, data['y_predict_C=0'].values), 3)),
          str(round(roc_auc_score(data.y.values, data['y_predict_C=10'].values), 3))]

In [16]:
auc

['0.927', '0.936']

In [17]:
write_answer('logistic_answer1', ' '.join(auc))

6) Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?   
**НЕ СХОДИТСЯ**   
**РАСТЕТ**

In [18]:
def grad_desc(data, w0=(0, 0), eps=1e-5, max_iter=10000, C=10, k=0.1):
    w_old = w0
    n_iter = 0
    delta = 2 * eps
    while (delta > eps and n_iter < max_iter):
        w_new = step(data, w_old, C=C, k=k)
        delta = np.sqrt((w_new[0] - w_old[0])**2 + (w_new[1] - w_old[1])**2)
        w_old = w_new
        n_iter += 1
        
    return (w_new, n_iter, k)

In [19]:
%%time
w_n_k  = grad_desc(data, w0=(0, 0), eps=1e-5, max_iter=10000, C=10, k=0.5)
print(w_n_k)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys


((nan, nan), 515, 0.5)
Wall time: 21.5 s


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in double_scalars
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [20]:
%%time
w_n_k  = grad_desc(data, w0=(0, 0), eps=1e-5, max_iter=10000, C=10, k=0.1)
print(w_n_k)

((0.028558754546234223, 0.024780137249735559), 8, 0.1)
Wall time: 326 ms


In [21]:
%%time
w_n_k  = grad_desc(data, w0=(0, 0), eps=1e-5, max_iter=10000, C=10, k=0.01)
print(w_n_k)

((0.028497257934787239, 0.024751582489814787), 47, 0.01)
Wall time: 1.88 s


7) Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?   
**НЕТ**

In [22]:
%%time
w_n_k  = grad_desc(data, w0=(1, 1), eps=1e-5, max_iter=10000, C=10, k=0.1)
print(w_n_k)

((0.02855982899417691, 0.024781314495601135), 10, 0.1)
Wall time: 398 ms


In [23]:
%%time
w_n_k  = grad_desc(data, w0=(10, 10), eps=1e-5, max_iter=10000, C=10, k=0.1)
print(w_n_k)

((0.028559872377880276, 0.024781362030089259), 10, 0.1)
Wall time: 399 ms
